First we define a simple pipeline of two agents, of which one will generate a signal 
(in our case a *SineGeneratorAgent*) and the other one plots the signal on the 
dashboard (this is always a *MonitorAgent*).

We define a *SineGeneratorAgent* for which we have to override the 
functions `init_parameters()` & `agent_loop()` to define the new agent's behaviour.

*   `init_parameters()` is used to setup the input data stream and potentially other 
necessary parameters.
*   `agent_loop()` will be endlessly repeated until further notice. It will 
sample by sample extract the input data stream's content and push it to all agents 
connected to *SineGeneratorAgent*'s output channel by invoking `send_output()`.

The *MonitorAgent* is connected to the *SineGeneratorAgent*'s output channel and per 
default automatically plots the output. 

Each agent has an internal `current_state` which can be used as a switch to change the 
behaviour of the agent. The available states are listed
[here](https://github.com/bangxiangyong/agentMET4FOF/blob/a95b788544e8cce1e0bb757184da8c6447e96927/agentMET4FOF/agents.py#L78).

As soon as all agents are initialized and the connections are set up, the agent 
network is started by accordingly changing all agents' state simultaneously.

In [6]:
%load tutorial_1_generator_agent.py